In [12]:
# src/gnn_model.py
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    roc_auc_score, precision_recall_curve, auc,
    accuracy_score, f1_score
)

# ============================= CONFIG =============================
DATA_PROCESSED = "/content"  # Colab path
RESULTS_DIR = "results"
MODELS_DIR = "models"
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

WINDOW_SIZE = 10
STRIDE = 5
BATCH_SIZE = 16
EPOCHS = 80
MAX_ATTACK_RATIO = 0.30

# ============================= GRAPH BUILDER =============================
def build_graphs_with_ratio(df: pd.DataFrame):
    graphs = []
    for start in range(0, len(df) - WINDOW_SIZE + 1, STRIDE):
        win = df.iloc[start:start + WINDOW_SIZE].copy()
        if len(win) < 8:
            continue

        win['src_node'] = win['service'].astype(str) + '_src'
        win['dst_node'] = win['service'].astype(str) + '_dst'

        nodes = pd.unique(win[['src_node', 'dst_node']].values.ravel())
        if len(nodes) < 2:
            continue

        node2idx = {n: i for i, n in enumerate(nodes)}

        edge_index = []
        edge_attr = []
        for _, r in win.iterrows():
            src = node2idx[r['src_node']]
            dst = node2idx[r['dst_node']]
            edge_index.append([src, dst])
            edge_attr.append([
                r['duration'], r['src_bytes'], r['dst_bytes'],
                r['count'], r['same_srv_rate'], r['diff_srv_rate'],
                r['serror_rate'], r['rerror_rate']
            ])

        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)

        node_feat = np.array([
            [
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['duration'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['src_bytes'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['dst_bytes'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['count'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['same_srv_rate'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['diff_srv_rate'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['serror_rate'].mean(),
                win[(win['src_node'] == n) | (win['dst_node'] == n)]['rerror_rate'].mean(),
                len(win[(win['src_node'] == n) | (win['dst_node'] == n)]),
                1.0 if 'http' in n else 0.0
            ] for n in nodes
        ], dtype=np.float32)
        x = torch.from_numpy(node_feat)

        attack_ratio = (win['label'] == 'attack').mean()
        y = 1 if attack_ratio > 0.5 else 0
        graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=torch.tensor([y]))
        graphs.append((graph, attack_ratio))
    return graphs

# ============================= MODEL =============================
class GNNAnomalyDetector(nn.Module):
    def __init__(self, input_dim, hidden=128, layers=3):
        super().__init__()
        self.convs = nn.ModuleList([GCNConv(input_dim, hidden)] +
                                   [GCNConv(hidden, hidden) for _ in range(layers-1)])
        self.lin1 = nn.Linear(hidden, hidden // 2)
        self.lin2 = nn.Linear(hidden // 2, 1)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.4)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.relu(x)
            x = self.drop(x)
        x = global_mean_pool(x, batch)
        x = self.relu(self.lin1(x))
        x = self.drop(x)
        return self.lin2(x).squeeze(-1)

# ============================= TRAINING =============================
def train_gnn():
    train_df = pd.read_csv(f"{DATA_PROCESSED}/train_processed.csv")
    test_df  = pd.read_csv(f"{DATA_PROCESSED}/test_processed.csv")

    normal_train = train_df[train_df['label'] == 'normal'].copy()
    train_graphs = [g for g, _ in build_graphs_with_ratio(normal_train)]

    test_graphs_with_ratio = build_graphs_with_ratio(test_df)
    test_graphs = [g for g, _ in test_graphs_with_ratio]

    normal_windows = [g for g, r in test_graphs_with_ratio if r <= MAX_ATTACK_RATIO]
    attack_graphs = [g for g in test_graphs if g.y.item() == 1]

    normal_windows = normal_windows[:50]
    attack_graphs = attack_graphs[:500]
    final_test_graphs = normal_windows + attack_graphs

    train_loader = DataLoader(train_graphs, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(final_test_graphs, batch_size=BATCH_SIZE, shuffle=False)

    input_dim = train_graphs[0].x.shape[1]
    model = GNNAnomalyDetector(input_dim).to(DEVICE)
    opt = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
    crit = nn.BCEWithLogitsLoss()

    print("Training...")
    for epoch in range(1, EPOCHS + 1):
        model.train()
        epoch_loss = 0.0
        for data in train_loader:
            data = data.to(DEVICE)
            data.y = torch.zeros(data.num_graphs, device=DEVICE)
            logits = model(data)
            loss = crit(logits, data.y)
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            epoch_loss += loss.item()
        if epoch % 20 == 0:
            avg = epoch_loss / len(train_loader)
            print(f"  Epoch {epoch:2d} | Loss: {avg:.8f}")

    torch.save(model.state_dict(), f"{MODELS_DIR}/gnn_detector.pth")

    # ============================= EVALUATION =============================
    model.eval()
    scores, y_true = [], []
    with torch.no_grad():
        for data in test_loader:
            data = data.to(DEVICE)
            logits = model(data)
            scores.extend(torch.sigmoid(logits).cpu().numpy())
            y_true.extend(data.y.cpu().numpy())

    scores = np.array(scores)
    y_true = np.array(y_true)

    # Threshold from training normal distribution
    normal_scores = []
    with torch.no_grad():
        for data in train_loader:
            data = data.to(DEVICE)
            normal_scores.extend(torch.sigmoid(model(data)).cpu().numpy())
    threshold = np.percentile(normal_scores, 95)

    # Convert scores to binary predictions
    y_pred = (scores > threshold).astype(int)

    # ===================== Added Metrics =====================
    auc_roc = roc_auc_score(y_true, scores)
    prec, rec, _ = precision_recall_curve(y_true, scores)
    auc_pr = auc(rec, prec)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    k = int(0.1 * len(scores))
    prec10 = np.mean(y_true[np.argsort(scores)[-k:]])

    print("\n📌 GNN Results (No Leakage)")
    print(f"ROC-AUC        : {auc_roc:.6f}")
    print(f"PR-AUC         : {auc_pr:.6f}")
    print(f"Accuracy       : {accuracy:.6f}")
    print(f"F1 Score       : {f1:.6f}")
    print(f"Precision@10%  : {prec10:.6f}")
    print(f"Threshold      : {threshold:.6f}")

    pd.DataFrame({"score": scores, "label": y_true}).to_csv(
        f"{RESULTS_DIR}/gnn_scores.csv", index=False
    )

    plt.figure(figsize=(10,6))
    sns.histplot(scores[y_true==0], label="Normal", alpha=0.7, bins=40)
    sns.histplot(scores[y_true==1], label="Attack", alpha=0.7, bins=40)
    plt.axvline(threshold, color='k', ls='--', label=f'Threshold={threshold:.4f}')
    plt.title("GNN Anomaly Score Distribution")
    plt.xlabel("Score"); plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/gnn_score_distribution.png", dpi=200)
    plt.close()

    print("\nResults saved in:", RESULTS_DIR)

# ============================= RUN =============================
train_gnn()

Using device: cuda
Training...
  Epoch 20 | Loss: 0.00000188
  Epoch 40 | Loss: 0.00000183
  Epoch 60 | Loss: 0.00000196
  Epoch 80 | Loss: 0.00000191

📌 GNN Results (No Leakage)
ROC-AUC        : 0.834920
PR-AUC         : 0.979943
Accuracy       : 0.783636
F1 Score       : 0.869374
Precision@10%  : 1.000000
Threshold      : 0.000003

Results saved in: results
